In [79]:
import pandas as pd

In [80]:
!pip install pyarrow fastparquet


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [81]:
parquet_file = "data/eco2mix-regional-cons-def.parquet"
df = pd.read_parquet(parquet_file)

csv_file = "data/eco2mix-regional-cons-def.csv"
df.to_csv(csv_file, index=False)

In [82]:
df.head()

,Code INSEE région,Région,Nature,Date,Heure,Date - Heure,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),...,TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%),Column 30
0,27,Bourgogne-Franche-Comté,Données définitives,2013-01-01,00:00,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
1,28,Normandie,Données définitives,2013-01-01,00:00,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,53,Bretagne,Données définitives,2013-01-01,00:00,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
3,32,Hauts-de-France,Données définitives,2013-01-01,00:00,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
4,75,Nouvelle-Aquitaine,Données définitives,2013-01-01,00:00,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


In [83]:
df.describe()

,Code INSEE région,Date - Heure,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Pompage (MW),Bioénergies (MW),Ech. physiques (MW)
count,1.980288e+06,1980288,1.980276e+06,1.980276e+06,1.155161e+06,1.980180e+06,1.980276e+06,1.980276e+06,1.120121e+06,1.980276e+06,1.843380e+06
mean,4.991667e+01,2017-09-15 22:10:23.036649,4.497690e+03,3.783493e+02,6.232625e+03,2.564869e+02,9.196955e+01,6.001617e+02,-1.161740e+02,8.332810e+01,-4.633691e+02
min,1.100000e+01,2012-12-31 23:00:00,7.200000e+02,-8.300000e+01,-1.440000e+02,-6.000000e+00,-2.300000e+01,-6.000000e+00,-2.581000e+03,4.000000e+00,-1.221900e+04
25%,2.775000e+01,2015-05-10 09:52:30,2.789000e+03,2.400000e+01,3.598000e+03,2.700000e+01,0.000000e+00,5.000000e+00,-1.600000e+01,4.800000e+01,-3.802000e+03
50%,4.800000e+01,2017-09-15 21:45:00,4.111000e+03,1.560000e+02,6.214000e+03,1.050000e+02,1.000000e+00,5.700000e+01,0.000000e+00,7.400000e+01,8.800000e+01
75%,7.525000e+01,2020-01-23 10:37:30,5.741000e+03,4.910000e+02,8.722000e+03,2.990000e+02,6.700000e+01,8.790000e+02,0.000000e+00,1.160000e+02,2.141000e+03
max,9.300000e+01,2022-05-31 21:30:00,1.533800e+04,4.293000e+03,1.363200e+04,4.900000e+03,2.894000e+03,8.742000e+03,0.000000e+00,5.480000e+02,1.395900e+04
std,2.564001e+01,NaN,2.170361e+03,5.407519e+02,3.043245e+03,4.333763e+02,2.247761e+02,1.028933e+03,3.134738e+02,4.462511e+01,4.209660e+03


In [84]:
df.dtypes

Code INSEE région               int64
Région                         object
Nature                         object
Date                           object
Heure                          object
Date - Heure           datetime64[us]
Consommation (MW)             float64
Thermique (MW)                float64
Nucléaire (MW)                float64
Eolien (MW)                   float64
Solaire (MW)                  float64
Hydraulique (MW)              float64
Pompage (MW)                  float64
Bioénergies (MW)              float64
Ech. physiques (MW)           float64
Stockage batterie              object
Déstockage batterie            object
Eolien terrestre               object
Eolien offshore                object
TCO Thermique (%)              object
TCH Thermique (%)              object
TCO Nucléaire (%)              object
TCH Nucléaire (%)              object
TCO Eolien (%)                 object
TCH Eolien (%)                 object
TCO Solaire (%)                object
TCH Solaire 

We are removing unnecessary data, specifically columns with a null value rate higher than 70%, as well as the Date and Heure (Time) fields, which are redundant with the Date - Heure (Time) field.

In [85]:
summarize = df.isnull().mean() * 100
columns_to_drop = summarize[summarize > 70].index.tolist()

columns_to_drop.extend(["Date", "Heure"])

df = df.drop(columns=columns_to_drop)

df

,Code INSEE région,Région,Nature,Date - Heure,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Pompage (MW),Bioénergies (MW),Ech. physiques (MW)
0,27,Bourgogne-Franche-Comté,Données définitives,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28,Normandie,Données définitives,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,53,Bretagne,Données définitives,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32,Hauts-de-France,Données définitives,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,75,Nouvelle-Aquitaine,Données définitives,2012-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980283,52,Pays de la Loire,Données consolidées,2022-05-31 21:30:00,721.0,5.0,NaN,162.0,4.0,1.0,NaN,54.0,NaN
1980284,84,Auvergne-Rhône-Alpes,Données consolidées,2022-05-31 21:30:00,6371.0,345.0,8558.0,12.0,1.0,4017.0,-6.0,106.0,NaN
1980285,44,Grand Est,Données consolidées,2022-05-31 21:30:00,4525.0,1220.0,3724.0,585.0,0.0,1086.0,0.0,122.0,NaN
1980286,75,Nouvelle-Aquitaine,Données consolidées,2022-05-31 21:30:00,3692.0,28.0,2693.0,156.0,0.0,184.0,NaN,165.0,NaN


Renamme all columns in English

In [86]:
rename_columns = {
    "Code INSEE région": "Region INSEE Code",
    "Région": "Region",
    "Nature": "Nature",
    "Date - Heure": "Date - Time",
    "Consommation (MW)": "Consumption (MW)",
    "Thermique (MW)": "Thermal (MW)",
    "Nucléaire (MW)": "Nuclear (MW)",
    "Eolien (MW)": "Wind (MW)",
    "Solaire (MW)": "Solar (MW)",
    "Hydraulique (MW)": "Hydraulic (MW)",
    "Pompage (MW)": "Pumping (MW)",
    "Bioénergies (MW)": "Bioenergies (MW)",
    "Ech. physiques (MW)": "Physical Exchanges (MW)",
    "Date - Heure": "Date - Time"
}

df = df.rename(columns=rename_columns)

Adapt the Hour in UTC + 1 like in France

In [87]:
df["Date - Time"] = df["Date - Time"] + pd.Timedelta(hours=1)
df

,Region INSEE Code,Region,Nature,Date - Time,Consumption (MW),Thermal (MW),Nuclear (MW),Wind (MW),Solar (MW),Hydraulic (MW),Pumping (MW),Bioenergies (MW),Physical Exchanges (MW)
0,27,Bourgogne-Franche-Comté,Données définitives,2013-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28,Normandie,Données définitives,2013-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,53,Bretagne,Données définitives,2013-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32,Hauts-de-France,Données définitives,2013-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,75,Nouvelle-Aquitaine,Données définitives,2013-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980283,52,Pays de la Loire,Données consolidées,2022-05-31 22:30:00,721.0,5.0,NaN,162.0,4.0,1.0,NaN,54.0,NaN
1980284,84,Auvergne-Rhône-Alpes,Données consolidées,2022-05-31 22:30:00,6371.0,345.0,8558.0,12.0,1.0,4017.0,-6.0,106.0,NaN
1980285,44,Grand Est,Données consolidées,2022-05-31 22:30:00,4525.0,1220.0,3724.0,585.0,0.0,1086.0,0.0,122.0,NaN
1980286,75,Nouvelle-Aquitaine,Données consolidées,2022-05-31 22:30:00,3692.0,28.0,2693.0,156.0,0.0,184.0,NaN,165.0,NaN


Remove rows filled with NA, specifically rows that provide no meaningful information. 
These rows only contain the date, region, and time, which are not useful on their own.

In [88]:
columns_to_check = [
    "Consumption (MW)", "Thermal (MW)", "Nuclear (MW)", 
    "Wind (MW)", "Solar (MW)", "Hydraulic (MW)", 
    "Pumping (MW)", "Bioenergies (MW)", "Physical Exchanges (MW)"
]


df = df.dropna(subset=columns_to_check, how="all")

Ad a new column (Production (MW)) and calculating its value as the sum of other columns, replacing NaN with 0 using fillna()

In [89]:
df = df.copy()

df["Production (MW)"] = (
    df["Consumption (MW)"].fillna(0) +
    df["Thermal (MW)"].fillna(0) +
    df["Nuclear (MW)"].fillna(0) +
    df["Wind (MW)"].fillna(0) +
    df["Solar (MW)"].fillna(0) +
    df["Hydraulic (MW)"].fillna(0) +
    df["Pumping (MW)"].fillna(0) +
    df["Bioenergies (MW)"].fillna(0) +
    df["Physical Exchanges (MW)"].fillna(0)
)
df

,Region INSEE Code,Region,Nature,Date - Time,Consumption (MW),Thermal (MW),Nuclear (MW),Wind (MW),Solar (MW),Hydraulic (MW),Pumping (MW),Bioenergies (MW),Physical Exchanges (MW),Production (MW)
12,52,Pays de la Loire,Données définitives,2013-01-01 00:30:00,3595.0,127.0,NaN,182.0,0.0,0.0,NaN,32.0,3252.0,7188.0
13,11,Île-de-France,Données définitives,2013-01-01 00:30:00,9134.0,685.0,NaN,16.0,0.0,0.0,NaN,142.0,8289.0,18266.0
14,24,Centre-Val de Loire,Données définitives,2013-01-01 00:30:00,2476.0,90.0,9085.0,508.0,0.0,34.0,NaN,25.0,-7267.0,4951.0
15,44,Grand Est,Données définitives,2013-01-01 00:30:00,4943.0,319.0,9137.0,1109.0,0.0,1418.0,0.0,57.0,-7102.0,9881.0
16,93,Provence-Alpes-Côte d'Azur,Données définitives,2013-01-01 00:30:00,5570.0,537.0,NaN,12.0,0.0,1412.0,0.0,58.0,3548.0,11137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980283,52,Pays de la Loire,Données consolidées,2022-05-31 22:30:00,721.0,5.0,NaN,162.0,4.0,1.0,NaN,54.0,NaN,947.0
1980284,84,Auvergne-Rhône-Alpes,Données consolidées,2022-05-31 22:30:00,6371.0,345.0,8558.0,12.0,1.0,4017.0,-6.0,106.0,NaN,19404.0
1980285,44,Grand Est,Données consolidées,2022-05-31 22:30:00,4525.0,1220.0,3724.0,585.0,0.0,1086.0,0.0,122.0,NaN,11262.0
1980286,75,Nouvelle-Aquitaine,Données consolidées,2022-05-31 22:30:00,3692.0,28.0,2693.0,156.0,0.0,184.0,NaN,165.0,NaN,6918.0


Calculation of TCO (Total Contribution Percentage) attributes by energy source (as a percentage relative to the total instantaneous global production).

In [90]:
df = df.copy()

df.loc[:, "TCO Thermal (MW)"] = (df["Thermal (MW)"] / df["Production (MW)"] * 100).round(2)
df.loc[:, "TCO Nuclear (MW)"] = (df["Nuclear (MW)"] / df["Production (MW)"] * 100).round(2)
df.loc[:, "TCO Wind (MW)"] = (df["Wind (MW)"] / df["Production (MW)"] * 100).round(2)
df.loc[:, "TCO Solar (MW)"] = (df["Solar (MW)"] / df["Production (MW)"] * 100).round(2)
df.loc[:, "TCO Hydraulic (MW)"] = (df["Hydraulic (MW)"] / df["Production (MW)"] * 100).round(2)

tco_columns = [
    "TCO Thermal (MW)", "TCO Nuclear (MW)", "TCO Wind (MW)", 
    "TCO Solar (MW)", "TCO Hydraulic (MW)"
]
df.loc[:, tco_columns] = df[tco_columns].fillna(0)

df


,Region INSEE Code,Region,Nature,Date - Time,Consumption (MW),Thermal (MW),Nuclear (MW),Wind (MW),Solar (MW),Hydraulic (MW),Pumping (MW),Bioenergies (MW),Physical Exchanges (MW),Production (MW),TCO Thermal (MW),TCO Nuclear (MW),TCO Wind (MW),TCO Solar (MW),TCO Hydraulic (MW)
12,52,Pays de la Loire,Données définitives,2013-01-01 00:30:00,3595.0,127.0,NaN,182.0,0.0,0.0,NaN,32.0,3252.0,7188.0,1.77,0.00,2.53,0.00,0.00
13,11,Île-de-France,Données définitives,2013-01-01 00:30:00,9134.0,685.0,NaN,16.0,0.0,0.0,NaN,142.0,8289.0,18266.0,3.75,0.00,0.09,0.00,0.00
14,24,Centre-Val de Loire,Données définitives,2013-01-01 00:30:00,2476.0,90.0,9085.0,508.0,0.0,34.0,NaN,25.0,-7267.0,4951.0,1.82,183.50,10.26,0.00,0.69
15,44,Grand Est,Données définitives,2013-01-01 00:30:00,4943.0,319.0,9137.0,1109.0,0.0,1418.0,0.0,57.0,-7102.0,9881.0,3.23,92.47,11.22,0.00,14.35
16,93,Provence-Alpes-Côte d'Azur,Données définitives,2013-01-01 00:30:00,5570.0,537.0,NaN,12.0,0.0,1412.0,0.0,58.0,3548.0,11137.0,4.82,0.00,0.11,0.00,12.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980283,52,Pays de la Loire,Données consolidées,2022-05-31 22:30:00,721.0,5.0,NaN,162.0,4.0,1.0,NaN,54.0,NaN,947.0,0.53,0.00,17.11,0.42,0.11
1980284,84,Auvergne-Rhône-Alpes,Données consolidées,2022-05-31 22:30:00,6371.0,345.0,8558.0,12.0,1.0,4017.0,-6.0,106.0,NaN,19404.0,1.78,44.10,0.06,0.01,20.70
1980285,44,Grand Est,Données consolidées,2022-05-31 22:30:00,4525.0,1220.0,3724.0,585.0,0.0,1086.0,0.0,122.0,NaN,11262.0,10.83,33.07,5.19,0.00,9.64
1980286,75,Nouvelle-Aquitaine,Données consolidées,2022-05-31 22:30:00,3692.0,28.0,2693.0,156.0,0.0,184.0,NaN,165.0,NaN,6918.0,0.40,38.93,2.25,0.00,2.66


In [91]:
df.to_parquet("data/eco2mix.parquet", engine="pyarrow", index=False)